In [1]:
from board import *
from PIL import Image
from collections import namedtuple
from typing import Tuple
from pathlib import Path

In [2]:
Coord = namedtuple('coord', 'file rank')
CropBox = Tuple[float, float, float, float]

In [3]:
def get_coord(name) -> Coord:
    """ name: 2 character coordinate of board square. A8, h3, f4, etc """
    file, rank = name.lower()[0], name.lower()[1]
    assert file in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
    assert rank in ['1', '2', '3', '4', '5', '6', '7', '8']
    return Coord(rank=rank, file=file)

In [4]:
def get_crop_box(coord: Coord, height: float, width: float) -> CropBox:
    """ 
    coord: board coordinate, a8, h3, f4, etc
    height: float of board image height
    width: float of board image width
    """
    
    origin_y = (8 - int(coord.rank)) * height / 8
    origin_x = int(ord(coord.file) - ord('a')) * width / 8
    return (origin_x, origin_y, origin_x + width / 8, origin_y + height / 8)

In [5]:
def get_image(coord, img):
    cropbox = get_crop_box(coord, img.height, img.width)
    return img.crop(cropbox)

## black
* king e8
* queen d8
* knight b8, g8
* rook a8, h8
* bishop c8, f8
* pawn a7-h7


## white
* king e1
* queen d1
* knight b1, g1
* rook a1, h1
* bishop c1, f1
* pawn a2-h2

## empty
* a3 - h6

## should i model the pieces as different colors, or no?

ultimately what's white and what's black is relative to the playing set. What is white in one set, could be black in another set where it's the darker color. So instead just classify the pieces, and then use relative coloring to figure out whether it's white or black on that set.


In [6]:
p = {'king': ['e1', 'e8'], 'queen': ['d1', 'd8'], 'knight': ['b1', 'g1', 'b8', 'g8'], 'rook': ['a1', 'h1', 'a8', 'h8'], 'bishop': ['c1', 'f1', 'c8', 'f8'], 'pawn': ['a2', 'b2', 'c2', 'd2', 'e2', 'f2', 'g2', 'h2', 'a7', 'b7', 'c7', 'd7', 'e7', 'f7', 'g7', 'h7']}

In [7]:
files = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
ranks = range(3, 7)
none = {'none': [f'{file}{rank}' for file in files for rank in ranks]}

In [8]:
piece_location = {**p, **none}

In [9]:
for piece in piece_location.keys():
    p = Path(f'./data/training/{piece}')
    if not p.exists():
        p.mkdir(mode=0o777, parents=True, exist_ok=False)

In [10]:
boards = Path('./data/_board')

In [12]:
for board in boards.glob("*.png"):
    img = Image.open(board)
    for piece, locations in piece_location.items():
        path = Path(f'./data/training/{piece}')
        for location in locations:
            files = list(path.glob('*.png'))
            if not files:
                new_file = '0'
            else:
                # todo: store the max for each piece instead of recalcing it each time
                new_file = int(sorted(list(path.glob('*png')), key=lambda x: int(x.stem))[-1].stem) + 1 # hack
                
            save_path = Path(f'./data/training/{piece}/{new_file}.png')
            coord = get_coord(location)
            piece_image = get_image(coord, img)
            piece_image.save(save_path)